# Extensions to Linear Models

## Interaction Terms and Polynomial Regression

In [ ]:
import pandas as pd 
import numpy as np 

import seaborn as sns 
%matplotlib inline 

from sklearn.linear_model import LinearRegression, Lasso, Ridge

from sklearn.preprocessing import PolynomialFeatures, StandardScaler

from sklearn.model_selection import train_test_split 

from sklearn.metrics import mean_squared_error

You've seen that when performing multiple linear regression, we are trying to find the best parameters $\beta_i$ to fit a model of the form 

$$\large  y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + ... + \beta_n x_n $$ 

to the data. 

By "best parameters", we mean those parameters that minimize the cost function for linear regression.  

Here $y$ is the dependent variable or target, $n$ is the number of independent variables or features of the model, $\beta_0$ is the intercept, and $\beta_i$ (where $i$ is equal to 1, 2, ..., n) is the coefficient that tells us by how much the target $y$ changes if $x_i$ increases by a unit, all other features staying constant. 

By construction, this model assumes that there is a linear relationship between the target variable and the features. 

**But what about times when features have a non-linear relationship to the data, or when features interact with each other?** 

### Interactions 
Interaction terms allow us to model relationships when the effects of a feature on the target is influenced by another feature. 

$$\large  y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_{1, 2} x_1 x_2 + ... + \beta_n x_n $$ 

### Polynomial terms 

Polynomial terms allow us to capture non-linear relationships between a feature and the target. 

We can use polynomial regression to capture non-linear relationships of features to the target. 

> This is still a linear regression problem because the regression function is linear in the unknown coefficients/parameters $\beta$ that are estimated from the data. 

$$\large  y = \beta_0 + \beta_1 x + \beta_2 x^2 + \beta_3 x^3 + ... $$

**We can use `scikit-learn`'s `PolynomialFeatures` to create polynomial and interaction terms from our features.**

We will see examples of this tool in action soon below. 

Next, let's discuss one of the most important concepts regarding model fitting: **the bias-variance tradeoff.** 

## Bias-Variance Tradeoff

### Bias and Variance

When fitting a model to data, there are two main sources of error: reducible and irreducible error. 

Reducible error is comprised of bias and variance. 

* **Bias** error is an error from incorrect assumptions in the model. When a model has too high bias it can miss relevant relationships between features and target and can cause a model to underfit and not be able to generalize to unseen data

* **Variance** is error due to sensitivity to variations in the training set. A model with too high variance trains on the noise rather than the signal, and also doesn't generalize to unseen data and overfits it. 

**Bias error measures how much the predicted values of your model differ from the true values.** Bias occurs when a model has limited flexibility to learn the _true_ signal from a dataset. 

**Variance error occurs when a model is very sensitive to specific sets of training data.** 

We can decompose the reducible error into bias and variance.

<img src="images/bias-variance-tradeoff.png" width=500>

(source: [The Bias-Variance Tradeoff](https://djsaunde.wordpress.com/2017/07/17/the-bias-variance-tradeoff/))

If your model has too few predictors or if you have too few data in your training set, it might suffer from **high bias** and underfit your data. 

If your model is too complex and has too many predictors, it might suffer from **high variance** and overfit your data. 

In both cases, the model won't generalize well to unseen data!  

We want to train models with _just enough_ number of features to minimize the model's generalization error.

One of the ways we can control model complexity is by using **Regularization**, which we discuss next. 

## Regularization

Suppose I have split my data into training and testing sets. Do I want my model to fit my training data _exactly_ ?

> No. A model like this will have too much variance (fitting the noise!) and overfit the data. It won't generalize to the unseen test data. 

Overfitting is generally a result of high variance. 

High variance can be caused by:
    
- having irrelevant or too many predictors

- multicollinearity

- large coefficients

The first problem is about picking up on noise rather than signal.

The second problem is about having a least-squares estimate that is highly sensitive to random error.

The third is about having highly sensitive predictors. 

### Regularization to the rescue! 

**Regularization** is about introducing a factor into our model's cost function designed to enforce the stricture that the coefficients stay small, by **penalizing** the coefficients that get too large.

This technique helps us avoid model overfitting, as it discourages "learning" a very complex model (by shrinking coefficients to smaller values or shrinking them to zero altogether!) 

To implement regularization techniques, we alter the linear regression __cost function__ such that the goal now is not merely to minimize the difference between actual values and our model's predicted values. 

* Rather, we add in a term to our loss function that represents the sizes of the coefficients.

### Lasso ("L1") Regularization: 

Modified cost function: 

$$\large\Sigma^{n_{obs.}}_{i=1}[(y_i - \Sigma^{n_{feat.}}_{j=0}\beta_j \times x_{ij})^2]+ \lambda\space\Sigma^{n_{feat.}}_{j=0}|\beta_j|$$
<br/> <br/>

### Ridge ("L2") Regularization: 

Modified cost function: 

$$\large\Sigma^{n_{obs.}}_{i=1}[(y_i - \Sigma^{n_{feat.}}_{j=0}\beta_j \times x_{ij})^2] + \lambda\space\Sigma^{n_{feat.}}_{j=0}\beta^2_j$$


**Don't let these formulas intimidate you!** 

The first term in each of these (the sum of squares) is the same, familiar cost function (also known as loss function) we use in linear regression! 

What distinguishes Lasso Regression from Ridge Regression is the additional term on the right: the penalty term!  

* The Lasso uses the absolute values of the coefficients.
    * As a consequence of the form of this penalty term, the Lasso can set the coefficient for a feature equal to zero. 
        * This is why the Lasso is used as a feature selection method. 
<br> </br>
* The Ridge uses the squares of the coefficients. 

For a nice discussion of these methods in Python, see https://towardsdatascience.com/ridge-and-lasso-regression-a-complete-guide-with-python-scikit-learn-e20e34bcbf0b.

## Scale the features before using Regularization!

When using regularization methods, we need to scale the features first! 

The terms added to the cost function in both Lasso and Ridge regression penalize large values of all coefficients **equally**. 

## Regularization: Example: Boston Housing Dataset

We'll use Lasso regression to fit a model to the Boston Housing Data using scikit-learn. 

In the cell below, we load the data for you and fit a baseline linear regression model to compare our Lasso regression model results to once we have them.

In [ ]:
from sklearn.datasets import load_boston

X, y = load_boston(return_X_y=True)

boston = load_boston()

df = pd.DataFrame(np.concatenate([X,y.reshape(-1,1)], axis=1), columns=boston.feature_names.tolist() + ['MEDV'])
df.head(1)

X = df.drop('MEDV', axis=1)
y = df['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)


# Train a baseline linenar regression model to the data
lr = LinearRegression()
lr.fit(X_train, y_train)

print('Training score: {}'.format(lr.score(X_train, y_train)))
print('Test score: {}'.format(lr.score(X_test, y_test)))

y_pred = lr.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
print('Training MSE: {}'.format(mse))

y_pred = lr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print('Test MSE: {}'.format(mse))

Let's first attempt to improve upon this model by adding polynomial features and interaction terms. 

We create the polynomial and interaction terms and fit the model for you. 

Evaluate the model performance on the test data. 

In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)

lr2 = LinearRegression()
lr2.fit(X_train_poly, y_train)
print("Training score:", lr2.score(X_train_poly, y_train))

mse = mean_squared_error(y_train, lr2.predict(X_train_poly))
print("Training MSE:", mse)

In [ ]:
# Transform the test data 
X_test_poly = poly.transform(None)

# Print R-squared for the test data 
print("Test score:", lr2.score(None, None))

# Compute and print the mean squared error of the model on the test data 
mse = mean_squared_error(y_train, lr2.predict(None))

print('Test MSE: {}'.format(mse))

How does the performance of the trained model compare on the training vs test data? **Compare the two resulting mean squared errors.**

Let's use Lasso regularization using the polynomial features to see if we can fit a model that outperforms the baseline linear regression model we fit previously.

In [ ]:
# Scale the features first!

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create polynomial and interaction terms 
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_scaled_poly = poly.fit_transform(X_train_scaled)
X_test_scaled_poly = poly.transform(X_test_scaled)

In [ ]:
# Instantiate and fit a lasso model. Set alpha = 0.3 and random_state=1
lasso = Lasso(alpha=None, random_state=None)
lasso.fit(None, None)

In [ ]:
# Compute and print the R-squared of the model on the training data
print("Training score:", lasso.score(None, None))

# Compute and print the mean squared error of the model on the test data 
mse = mean_squared_error(y_train, lasso.predict(None))

print('Training MSE: {}'.format(mse))

In [ ]:
# Compute and print the R-squared of the model on the training data
print("Training score:", lasso.score(X_train_scaled_poly, y_train))

# Compute and print the mean squared error of the model on the test data 
mse = mean_squared_error(y_train, lasso.predict(X_train_scaled_poly))

print('Training MSE: {}'.format(mse))

In [ ]:
# Compute and print the R-squared of the model on the training data
print("Test score:", lasso.score(None, None))

# Compute and print the mean squared error of the model on the test data 
mse = mean_squared_error(None, lasso.predict(None))

print('Test MSE: {}'.format(mse))

Compare the performance of this model on the test data with the performance of the baseline model trained at the beginning. 

Did the Lasso set any coefficients to zero? 

# Summary: 

1. Interaction and polynomial terms can be added to linear models to account for non-linear relationships between the features and target, and to account for effects of a feature on the target when it is influenced by another feature. 

2. You can use `sklearn.preprocessing.PolynomialFeatures` to create interaction terms and polynomial terms. 

3. Error due to bias may occur when we use too few features in our model or when we don't have enough data. Errors due to variance occur when we fit a model that is too complex that overfits to the noise of the data and fails to generalize to unseen data.

4. We use regularization techniques like the Lasso and Ridge regression to reign in overfitting models by reducing their complexity. 

5. Before applying regularization techniques, you **must** scale the features. 

6. You can implement Lasso regularization in Python using `sklearn.linear_model.Lasso`. 


# Optional

## Example: Interaction and polynomial terms

The `Advertising` dataset has data on the number of sales of a product in 200 different markets, with information about how much money was spent on advertising on TV, radio, and on the newspaper. 

We want to know how advertising on these different media effects number of sales of the product. 

In the cell below, we load the `Advertising` dataset for you. 

In [ ]:
df = pd.read_csv('Advertising.csv')

# Look at the first five rows of data
df.head(1)

In [ ]:
# Let's drop the first column; it's meaningless
df.drop('Unnamed: 0', axis=1, inplace=True)

We will fit a baseline linear regression model to the data. 

* We first make sure to split the data into training and test sets.

In [ ]:
X = df[['TV', 'radio', 'newspaper']]
y = df['sales']

# Split the data into training and test sets. 
# Use random_state = 1 for reproducibility of results. 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1) 

* Next, we instantiate a `LinearRegression` model and fit the model to the training data 

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

Next, we want to assess model performance. 

To start, we compute the model's coefficient of determination, $R^2$ on the training and test data. 

In [ ]:
r2 = lr.score(X_train, y_train)
print("Training set R-squared:", r2)

r2 = lr.score(X_test, y_test)
print("Test set R-squared:", r2)

Next, we look at the model's adjusted coefficient of determination, $R_{adj}^{2}$ on the training and test data. 

In [ ]:
def adjusted_r2(X, y, fitted_model):
    r2 = fitted_model.score(X, y)
    n = X.shape[0]
    k = X.shape[1]
    adj_r2 = 1 - ((1-r2)*(n-1))/(n - k - 1)
    return adj_r2

print("Training set Adjusted R-squared", adjusted_r2(X_train, y_train, lr))
print("Test set Adjusted R-squared", adjusted_r2(X_test, y_test, lr))

Finally, we look at the model's mean squared error on the training and test data. 

In [ ]:
mse_training = mean_squared_error(y_train, lr.predict(X_train))
print("Training set MSE:", mse_training)

mse_test = mean_squared_error(y_test, lr.predict(X_test))
print("Test set MSE:", mse_test)

One way we can attempt to improve on this model is to include interaction terms. 

Maybe spending the advertising budget on one media, like newspaper, increases the effectiveness of TV advertising. In marketing, they call this synergy effects. We call this interactions. 

Let's add this interaction term to our data and fit a linear regression model that takes this into account.

In [ ]:
# Add interaction term between TV and newspaper
df['TV*newspaper'] = df['TV']*df['newspaper']

# Inspect the data
df.head(1)

**Fit a linear regression model to predict sales using the `TV`, `radio`, `newspaper`, and `TV*newspaper` features.** 

_Hint: Follow the steps that we used above to fit the linear regression model_

In [ ]:
# Split data into training and test sets. Use random_state=1. 

# your code here 

# Instantiate a LinearRegression object and fit it to the training data 

# your code here 

# Compute the trained model's performance on the training and test data:

# R2 
r2 = None
print("Training set R-squared:", r2)

r2 = None 
print("Test set R-squared:", r2)

# Adjusted R2 
print("Training set Adjusted R-squared", None)
print("Test set Adjusted R-squared", None)

# Mean Squared Error 
mse_training = None 
print("Training set MSE:", mse_training)

mse_test = None 
print("Test set MSE:", mse_test)

**Did the model performance change?** 

Let's take a quick look at the original data using `sns.pairplot`. 

In [ ]:
sns.pairplot(df.drop(['TV*newspaper'], axis=1))

What if we wanted to know about other possible interactions between the features and their effect on `sales`?

Also, from our pair plot above, we see that the relation between the features and the target is not exactly linear. 

We can use `sklearn.preprocessing.PolynomialFeatures` to generate polynomial and interaction features. 

Let's see what fitting a model with all these features does for our model performance! 

To create the polynomial and interaction terms, run the cell below. 

We are only interested in creating terms with degree 2. This means we care about creating features like the square of the TV advertising budget or interaction terms like TV budget times newspaper budget. 

In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(df[['TV','radio','newspaper']])
X_poly = pd.DataFrame(X_poly, columns=poly.get_feature_names(['TV','radio','newspaper']))

Next, you'll fit a linear regression model and assess model performance on the training and test data. 

You know the drill! (See the examples above.)

In [ ]:
# Split data into training and test sets. Use random_state=1. 

# your code here 

# Instantiate a LinearRegression object and fit it to the training data 

# your code here 

# Compute the trained model's performance on the training and test data:

# R2 
r2 = None
print("Training set R-squared:", r2)

r2 = None 
print("Test set R-squared:", r2)

# Adjusted R2 
print("Training set Adjusted R-squared", None)
print("Test set Adjusted R-squared", None)

# Mean Squared Error 
mse_training = None 
print("Training set MSE:", mse_training)

mse_test = None 
print("Test set MSE:", mse_test)

**Did the model performance change?**

## Ridge regression 

Repeat the Boston housing dataset, this time using ridge regression instead of the Lasso. 